# Chap05 - 오히려 큰 손해를 보시겠습니까?

# Would you rather lose an arm or a leg?

## 5.1 서론

> 어느 기상학자가 허리케인이	그가	사는	도시를	덮칠	확률을	예측하고	있다.	그는	95%	신뢰수준으로	허리케인이	
도시를	덮치지	않을	확률이	99~100%	사이라고	추정한다.	기상학자는	자신의	정확도에	매우	만족하며	재난	대피는	필요	없다고	시청에	권고하였다.	그러나	불행히도	허리케인은	도시를	덮쳤고,	도시는	물에	잠겼다.


> ***"정확하게 틀리는 것보다 대강 맞는 것이 더 낫다."***

## 5.2 손실함수 - Loss Functions

손실함수(loss function)에 대해 알아보자. 손실함수는 진짜(true) 모수의 함수이며 그 모수의 추정치다.

$$
L(\theta, \hat{\theta} ) = f(\theta, \hat{\theta})
$$

손실함수에서 중요한 점은 현재 추정치가 얼마나 나쁜지를 측정한다는 것이다. 손실함수의 값이 클수록, 추정치는 손실함수에 따라 더 나빠진다. 일반적인 손실함수의 예로는 **제곱오차손실(squared-error loss)** 이다.

$$
L\left(\theta, \hat{\theta}\right) = \left( \theta - \hat{\theta} \right)^{2}
$$

또한, 다음과 같이 비대칭 제곱오차손실함수(asymmetric squared-error loss function)도 있다.

$$
L\left(\theta, \hat{\theta} \right)=\begin{cases} \left(\theta - \hat{\theta} \right)^{2} \quad \hat{\theta} < \theta \\ c\left( \theta - \hat{\theta} \right)^{2} \quad \hat{\theta} \ge \theta, 0<c<1  \end{cases}
$$

위의 식은 진짜 모수보다 큰 값을 추정하는 것이 작은 값을 추정하는 것보다 낫다는 것을 보여준다. 예를 들어, 다음 달의 웹 트래픽을 추정하려는 경우 사용할 수 있는데, 여유있게 예상하여 서버 자원이 부족하게 배치되는 것을 피할 수 있기 때문이다.

**절대손실**(absolute-loss)함수는 오차가 선형적으로 증가하는 손실함수이다. 

$$
L \left( \theta, \hat{\theta} \right) = \left| \theta - \hat{\theta} \right|
$$

손실함수는 (1) 수학적 용이성과 (2) 적용할 때 결과값이 크게 영향을 받지 않는다.

### 5.2.1 현실 세계에서의 손실함수

베이지안 추론에서는 추론하고자 하는 (진짜) 모수가 사전/사후확률분포를 가진 확률변수라고 생각한다. 사후확률분포에서 뽑은 값은 모수의 참값이 될 수 있는 가능성이 있는 값이다. 따라서, 이러한 사후확률분포에서 **기대손실**(expected loss)를 계산한다. 기대손실은 사후확률분포에서 얻은 표본 한 개에 주어진 손실보다 참(true)손실에 대해 더 정확하게 추정한다.